# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm

import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

In [ ]:
!pip install speechpy
import speechpy


In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',   'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"danielkwizera","key":"858f28b48352b29cb72fa9f3815c1b5d"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# commands to download data from kaggle
!kaggle competitions download --force 11785-hw1p2-f23
!unzip -qo /kaggle/working/11785-hw1p2-f23.zip -d '/kaggle/working/'

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context =0, partition = "train-clean-100"):

        self.context = context
        self.phonemes = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root,partition,'mfcc/')
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(root,partition,'transcript')

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        length = len(mfcc_names)
        T = 0

        for i in range(length):
            #   Load a single mfcc
            mfcc  = np.load(os.path.join(self.mfcc_dir+mfcc_names[i]))
            # Extract the length of the mfcc
            T += mfcc.shape[0]

        self.mfccs = np.zeros((T + 2*self.context, 28),dtype = np.float32)
        self.transcripts = np.zeros((T, ), dtype = np.uint8)

        cy, cx = 0,self.context

        for i in range(length):
            #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir+mfcc_names[i]))
            #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = torch.from_numpy((mfcc - np.mean(mfcc,axis = 0)) / np.std(mfcc,axis = 0))
           #Added axis =0, as advised by the TA on Piazza
            #Watch out for an issue with the datatype, needs to be tensors not numpy arrays
            #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir, transcript_names[i]))[1:-1]
            # Remove [SOS] and [EOS] from the transcript
            #transcript = NotImplemented

            #  Convert transcript to a sequence of integers based on self.phonemes
            y = [self.phonemes.index(phoneme) for phoneme in transcript if phoneme in self.phonemes]

            T_i = mfcc.shape[0]
            self.mfccs[cx:cx + mfcc.shape[0]] = mfcc
            self.transcripts[cy:cy + mfcc.shape[0]] = y

            cx += mfcc.shape[0]
            cy += mfcc.shape[0]

        self.length = T

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind : ind + 2 * self.context + 1]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames =  torch.FloatTensor(frames)# Convert to tensors
        phonemes =  torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context =0, partition = "train-clean-100"):

        self.context = context
        self.phonemes = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root,partition,'mfcc/')
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        #self.transcript_dir = os.path.join(root,partition,'transcript')

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
       # transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
       # assert len(mfcc_names) == len(transcript_names)

        length = len(mfcc_names)
        T = 0

        for i in range(length):
            #   Load a single mfcc
            mfcc  = np.load(os.path.join(self.mfcc_dir+mfcc_names[i]))
            # Extract the length of the mfcc
            T += mfcc.shape[0]

        self.mfccs = np.zeros((T + 2*self.context, 28),dtype = np.float32)
        #self.transcripts = np.zeros((T, ), dtype = np.uint8)

        cx = self.context

        for i in range(length):
            #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir+mfcc_names[i]))
            #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = torch.from_numpy((mfcc - np.mean(mfcc,axis = 0)) / np.std(mfcc,axis = 0))
           #Added axis =0, as advised by the TA on Piazza
            #Watch out for an issue with the datatype, needs to be tensors not numpy arrays
            #   Load the corresponding transcript
            #transcript  = np.load(os.path.join(self.transcript_dir, transcript_names[i]))[1:-1]
            # Remove [SOS] and [EOS] from the transcript
            #transcript = NotImplemented

            #  Convert transcript to a sequence of integers based on self.phonemes
            #y = [self.phonemes.index(phoneme) for phoneme in transcript if phoneme in self.phonemes]

            T_i = mfcc.shape[0]
            self.mfccs[cx:cx + mfcc.shape[0]] = mfcc
           # self.transcripts[cy:cy + mfcc.shape[0]] = y

            cx += mfcc.shape[0]
           # cy += mfcc.shape[0]

        self.length = T

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind : ind + 2 * self.context + 1]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames =  torch.FloatTensor(frames)# Convert to tensors
       # phonemes =  torch.tensor(self.transcripts[ind])

        return frames#, phonemes

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [ ]:
config = {
    'epochs'        : 35,
    'batch_size'    : 8192,
    'context'       : 20,
    'init_lr'       : 1e-3,
    'architecture'  : 'very-low-cutoff',
    'dataset_root'  : '/kaggle/working/11-785-f23-hw1p2/'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(config["dataset_root"],
                          phonemes = PHONEMES,
                          context=config["context"],
                          partition= "train-clean-100")#NotImplemented

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(config["dataset_root"],
                          phonemes = PHONEMES,
                          context=config["context"],
                          partition= "dev-clean")#NotImplemented
# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(config["dataset_root"],
                          phonemes = PHONEMES,
                          context=config["context"],
                          partition= "test-clean")

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(1450,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(1450, output_size) 
        )


    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
#summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 25 million parameters for HW1 (including ensembles)

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from torch.nn.utils import clip_grad_norm_
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagationz

        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()
        # Clip gradients to prevent explosion
        #clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="c1d931bb4beb9786d38d65463356ea4f554817f9") #API Key is in your wandb account, under settings (wandb.ai/settings)

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

best_accuracy = 0


for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim= 1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions += [PHONEMES[j] for j in predicted_phonemes]# TODO

            ### Release memory
            del mfccs, predicted_phonemes, logits
            torch.cuda.empty_cache()

    return test_predictions

In [ ]:
predictions = test(model, test_loader)

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f23 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle